# Assignment: Classification

Classification refers to categorizing the given data into classes. For example,
- Given an image of hand-written character, identifying the character (multi-class classification)
- Given an image, annotating it with all the objects present in the image (multi-label classification)
- Classifying an email as spam or non-spam (binary classification)
- Classifying a tumor as benign or malignant and so on

In this assignment, we will be building a classifier to classify emails as spam or non-spam. We will be using the Kaggle dataset [Spam or Not Spam Dataset](https://www.kaggle.com/datasets/ozlerhakan/spam-or-not-spam-dataset?resource=download) for this task. 

**Note**: You cannot load any libraries other than the mentioned ones.




### Data pre-processing
The first step in every machine learning algorithm is to process the raw data in some meaningful representations. We will be using the [Bag-of-Words](https://towardsdatascience.com/a-simple-explanation-of-the-bag-of-words-model-b88fc4f4971) representation to process the text. It comprises of following steps:

- Process emails line-by-line to extract all the words.
- Replace extracted words by their stem (root) word. This is known as stemming and lematization.
- Remove stop words like and, or, is, am, and so on.
- Assign a unique index to each word. This forms the vocabulary.
- Represent each email as a binary vector of length equal to the size of the vocabulary such that the $i^{th}$ element of the vector is 1 iff the $i^th$ word is present in the email.

Here we provide you with the function signature along with the expected functionality. You are expected to complete them accordingly. 

In [1]:
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

vocabulary = []

# takes an email as an argument
# read email line-by-line and extract all the words
# return list of extracted words
def read_email(email):
    words = []
    words.extend(word_tokenize(email))
    # for word in words:
    #     print(word)
    return words

# takes a list of words as an argument
# replace each word by their stem word
# return list of stem words
def stemming(words):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    stem_words = [stemmer.stem(word) for word in lemmatized_words]
    return stem_words

# takes a list of stem-words as an argument
# remove stop words
# return list of stem words after removing stop words
def remove_stop_words(words):
    stem_no_stop_words = []
    for word in words:
        if word not in stopwords:
            stem_no_stop_words.append(word)
    return stem_no_stop_words

# takes a list of stem-words as an argument
# add new words to the vocabulary and assign a unique index to them
# returns new vocabulary
def build_vocabulary(words):
    for word in words:
        if(word not in vocabulary):
            vocabulary.append(word)
    vocab = vocabulary.copy()
    return vocab

# takes a list of stem-words and vocabulary as an argument
# returns bow representation
def get_bow(words, vocabWords):
    email_bow = [0]*len(vocabWords)
    for word in words:
        email_bow[vocabWords.index(word)] = 1
    return email_bow

# wrapper function
def text_to_bow(email):
    words = remove_stop_words(stemming(read_email(email)))
    bow = get_bow(words, build_vocabulary(words))
    return bow

# read the entire dataset
# convert emails to bow and maintain their labels
# call function text_to_bow()
def read_data():
    inputFileName = str(input("Please type the email file name you want to check for spam: "))
    data = np.genfromtxt(inputFileName, encoding = "utf8", dtype = "str", delimiter = ",")
    newData = []
    for i in range(0,len(data)):
        if i == 0:
            continue
        newData.append([text_to_bow(data[i][0]),data[i][1]])
    return data

read_data()

TypeError: argument of type 'LazyCorpusLoader' is not iterable

### Data Visualization
Let's understand the data distribution
- Visualize the frequency of word-occurence in all the emails(spam + non-spam)
- Visualize the freuency of word-occurence for spam and non-spam emails separately

In [ ]:
import matplotlib.pyplot as plt

# visuallze data distribution
def data_vis(data):
    # plt.style.use('_mpl-gallery')

    sum_spam = [0]*len(vocabulary)
    sum_non_spam = [0]*len(vocabulary)

    sum = np.sum(data[:][2],axis = 0)
    for line in data:
        if(line[1]==1):
            sum_spam += line[2]
        else:
            sum_non_spam += line[2]

    fig = plt.figure(figsize = (10, 5))
    plt.bar(vocabulary, sum, color ='blue', width = 0.4)
    plt.xlabel("Vocabulary")
    plt.ylabel("Frequency")
    plt.title("Data Visualization of frequency of word-occurrence in all the emails")
    plt.show()

    fig = plt.figure(figsize = (10, 5))
    plt.bar(vocabulary, sum_spam, color ='red', width = 0.4)
    plt.xlabel("Vocabulary")
    plt.ylabel("Frequency")
    plt.title("Data Visualization of frequency of word-occurrence in spam emails")
    plt.show()

    fig = plt.figure(figsize = (10, 5))
    plt.bar(vocabulary, sum_non_spam, color ='green', width = 0.4)
    plt.xlabel("Vocabulary")
    plt.ylabel("Frequency")
    plt.title("Data Visualization of frequency of word-occurrence in non-spam emails")
    plt.show()

    fig, ax = plt.subplots(figsize =(16, 9))
    ax.barh(vocabulary, sum)
    for s in ['top', 'bottom', 'left', 'right']:
        ax.spines[s].set_visible(False)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_tick_params(pad = 5)
    ax.yaxis.set_tick_params(pad = 10)
    ax.invert_yaxis()
    for i in ax.patches:
        plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
    ax.set_title('Frequency of words-occurrence in all the mails',
             loc ='left', )
    plt.show()

    return

data_vis(read_data())

### Learn a Classifier
Split the dataset randomly in the ratio 80:20 as the training and test dataset. Use only training dataset to learn the classifier. No test data should be used during training. Test data will only be used during evaluation.

Now let us try to use ML algorithms to classify emails as spam or non-spam. You are supposed to implement [SVM](https://scikit-learn.org/stable/modules/svm.html) and [K-Nearest Neighbour](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) algorithm available in scikit-learn using the same training dataset for both.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# split dataset
def split(data):
    mask = np.random.rand(len(data)) <= 0.8
    train_data = data[mask]
    test_data = data[~mask]
    return train_data, test_data

# learn a SVM model
# use the model to make prediction
# return the model predictions on train and test dataset
def svm_classifier(train_data, test_data):
    emails = train_data[:][0]
    label = train_data[:][1]
    clf = svm.SVC()
    clf.fit(emails, label)
    predict_labels = clf.predict(test_data[:][0])
    return predict_labels

# implement k-NN algorithm
# use the model to make prediction
# return the model predictions on train and test dataset
def knn_classifier(train_data, test_data):
    emails = train_data[:][0]
    label = train_data[:][1]
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(emails, label)
    predict_labels = neigh.predict(test_data[:][0])
    return predict_labels

train_data, test_data = split(read_data())
svm_train_predictions, svm_test_predictions = svm_classifier(train_data, test_data)
knn_train_predictions, knn_test_predictions = knn_classifier(train_data, test_data)

### Model Evaluation
Compare the SVM and k-NN model using metrics
- Accuracy
- [AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html)


In [ ]:
from sklearn import metrics

# compute accuracy 
def compute_accuracy(true_labels, predicted_labels):
    acc = metrics.accuracy_score(true_labels, predicted_labels)
    return acc

# compute AUC score 
def compute_auc(true_labels, predicted_labels):
    auc = metrics.roc_auc_score(true_labels, predicted_labels)
    return auc

# write code to print train and test accuracy and AUC score of SVM and k-NN classifier
def print_metrics(true_labels, predicted_labels,model,dataType):
    print(f"Accuracy of {model} for {dataType}: {compute_accuracy(true_labels, predicted_labels)}%")
    print(f"Area Under the Curve of {model} for {dataType}: {compute_auc(true_labels, predicted_labels)}")

print_metrics(train_data[:][1],svm_test_predictions,"SVM","train data")
print_metrics(train_data[:][1],knn_test_predictions,"KNN","train data")
print_metrics(test_data[:][1],svm_test_predictions,"SVM","test data")
print_metrics(test_data[:][1],knn_test_predictions,"KNN","test data")

In [ ]:
import numpy as np

arr = np.array([0,1])
brr = np.array([0,1])
crr = np.vstack([arr,brr])
crr